<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align="center"><font size="5">Classification with Python</font></h1>


In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:


In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset


This dataset is about past loans. The **Loan_train.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
| -------------- | ------------------------------------------------------------------------------------- |
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |


Lets download the dataset


In [ ]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

### Load Data From CSV File


In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object


In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing


Let’s see how many of each class is in our data set 


In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:


In [ ]:
# notice: installing seaborn might takes a few minutes
# !conda install -c anaconda seaborn -y

In [ ]:
# import seaborn as sns

# bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
# g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
# g.map(plt.hist, 'Principal', bins=bins, ec="k")

# g.axes[-1].legend()
# plt.show()

In [ ]:
# bins = np.linspace(df.age.min(), df.age.max(), 10)
# g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
# g.map(plt.hist, 'age', bins=bins, ec="k")

# g.axes[-1].legend()
# plt.show()

# Pre-processing:  Feature selection/extraction


### Lets look at the day of the week people get the loan


In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
# bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
# g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
# g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
# g.axes[-1].legend()
# plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 


In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values


Lets look at gender:


In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding

#### How about education?


In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding


In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection


Lets defind feature sets, X:


In [ ]:
X = Feature
X[0:5]

What are our lables?


In [ ]:
# To get numerical instead of categorical. Not necessary though.
# y = pd.Series(df['loan_status'], dtype="category").cat.codes.values

y = df['loan_status'].values
y[0:5]

## Normalize Data


Data Standardization give data zero mean and unit variance (technically should be done after train test split )


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification


Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:

-   K Nearest Neighbor(KNN)
-   Decision Tree
-   Support Vector Machine
-   Logistic Regression

** Notice:** 

-   You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
-   You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
-   You should include the code of the algorithm in the following cells.


# K Nearest Neighbor(KNN)

Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the **loan_test.csv** for finding the best k, however, you can split your train_loan.csv into train and test to find the best **k**.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

### Comments:

*test_size* parameter has an impact in final accuracy, as well as *the random_state*.
However, smaller *test_size* than 0.2 is not recommended as it probably overfits the model, despite the better final accuracy (0.82).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

Ks = 11
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range (1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    y_hat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, y_hat)
    std_acc[n-1]=np.std(y_hat==y_test)/np.sqrt(y_hat.shape[0])

mean_acc

print( "KNN - Best model accuracy=", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.fill_between(range(1,Ks),mean_acc - 3 * std_acc,mean_acc + 3 * std_acc, alpha=0.10,color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

In [ ]:
# Best K found to be 7. Train the final model with the optimal hyperparameters.

final_knn_model = KNeighborsClassifier(n_neighbors = 7)
final_knn_model.fit(X_train,y_train)

y_hat = final_knn_model.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_knn_model.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_hat))

# Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier

max_depth = 11
mean_acc = np.zeros((max_depth-1))
std_acc = np.zeros((max_depth-1))

# Grid-search over max_depth hyperparameter
for depth in range (1, max_depth):
    
    #Train Model and Predict
    clsTree = DecisionTreeClassifier(criterion="entropy", max_depth = depth)
    clsTree.fit(X_train,y_train)
    y_hat = clsTree.predict(X_test)
    mean_acc[depth-1] = metrics.accuracy_score(y_test, y_hat)
    std_acc[depth-1]=np.std(y_hat==y_test)/np.sqrt(y_hat.shape[0])

mean_acc

In [ ]:
plt.plot(range(1, max_depth), mean_acc, 'g')
plt.fill_between(range(1, max_depth), mean_acc - 1 * std_acc, mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Max Depth')
plt.tight_layout()
plt.show()

### Comments:

Analyzing the plot, we can see that for **max_depth=2** we achieve the best accuracy. However, defining a rather small value like max_depth=2 makes the DecisionTree not to be quite robust, as it will not be able to learn more complex patterns in data. Likewise, with larger values the model will probably overfit. The best value for the max_depth can be found after the the extreme curve in plot, which in our case is **max_depth=6**.
More info: https://www.kdnuggets.com/2019/08/understanding-decision-trees-classification-python.html

In [ ]:
# <!> NOTE: Better ommit visualization, !conda install will probably hang the kernel <!>

# Notice: You might need to uncomment and install the pydotplus and graphviz libraries if you have not installed these before
# !conda install -c conda-forge pydotplus -y
# !conda install -c conda-forge python-graphviz -y

# Maybe its better to try with pip
# !pip install graphviz
# !pip install pydotplus

# from  io import StringIO
# import pydotplus
# import matplotlib.image as mpimg
# from sklearn import tree

# dot_data = StringIO()
# filename = "class_tree.png"
# featureNames = Feature.columns
# out=tree.export_graphviz(class_tree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_train), filled=True,  special_characters=True,rotate=False)  
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# graph.write_png(filename)
# img = mpimg.imread(filename)
# plt.figure(figsize=(100, 200))
# plt.imshow(img,interpolation='nearest')

In [ ]:
# Best max_depth found to be 6. Train the final model with the optimal hyperparameters.

final_tree_model = DecisionTreeClassifier(criterion="entropy", max_depth = 6)
final_tree_model.fit(X_train,y_train)

y_hat = final_tree_model.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_tree_model.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_hat))

# Support Vector Machine


In [ ]:
import scipy.optimize as opt
from sklearn import svm
from sklearn.metrics import f1_score, jaccard_score

mean_acc = {}

# Grid-search over kernel hyperparameter
for kernel in ['linear', 'rbf', 'sigmoid', 'poly']:
    
    #Train Model and Predict
    clf = svm.SVC(kernel=kernel, C=0.75)
    clf.fit(X_train, y_train)
    y_hat = clf.predict(X_test)
    mean_acc[kernel] = metrics.accuracy_score(y_test, y_hat)

mean_acc

In [ ]:
# Best kernel found to be 'rbf'. Train the final model with the optimal hyperparameters.

final_svm_model = svm.SVC(kernel='rbf')
final_svm_model.fit(X_train, y_train)

y_hat = final_svm_model.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_tree_model.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_hat))
# jaccard_score(y_test, y_hat, pos_label='PAIDOFF')

# Logistic Regression


In [ ]:
import scipy.optimize as opt
from sklearn.linear_model import LogisticRegression

mean_acc = {}

# Grid-search over kernel and C (regularazitaion) hyperparameters
for slvr in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
#     for c in [1.0, 0.5, 0.3, 0.1, 0.01]:
    for c in [0.01]:
    
        #Train Model and Predict
        lr_clf = LogisticRegression(C=c, solver=slvr)
        lr_clf.fit(X_train, y_train)
        y_hat = lr_clf.predict(X_test)
        mean_acc[slvr + '-' + str(c)] = metrics.accuracy_score(y_test, y_hat)

dict(sorted(mean_acc.items(), key=lambda item: item[1], reverse=True))

### Comment:

Most solvers achieved the same score. More training data would help to decide about the best one, however we perform the final tests with the "newton-cg" solver.

In [ ]:
# Best hyperparameters found to be 'newton-cg' with C=0.01. Train the final model with the optimal hyperparameters.

final_regr_model = LogisticRegression(C=0.01, solver='newton-cg')
final_regr_model.fit(X_train, y_train)

y_hat = final_regr_model.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_regr_model.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_hat))

# Model Evaluation using Test set


In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:


In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation


In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
# Prepare test data like we did with train/test previously

test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)

test_df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df.groupby(['education'])['loan_status'].value_counts(normalize=True)

Feature = test_df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)

X_test = Feature
y_test = test_df['loan_status'].values
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)

In [ ]:
# Compute the scores using the final (best) trained models

# Final KNN model
y_hat_knn = final_knn_model.predict(X_test)
knn_f1 = f1_score(y_test, y_hat_knn, average='weighted') 
knn_jac = jaccard_score(y_test, y_hat_knn, pos_label='PAIDOFF')

print('KNN: F1 score: ', knn_f1)
print('KNN: Jaccard score: ', knn_jac)

# Final DecisionTree model
y_hat_tree = final_tree_model.predict(X_test)
tree_f1 = f1_score(y_test, y_hat_tree, average='weighted')
tree_jac = jaccard_score(y_test, y_hat_tree, pos_label='PAIDOFF')

print('\nDecisionTree: F1 score: ', tree_f1)
print('DecisionTree: Jaccard score: ', tree_jac)

# Final SVM model
y_hat_svm = final_svm_model.predict(X_test)
svm_f1 = f1_score(y_test, y_hat_svm, average='weighted')
svm_jac = jaccard_score(y_test, y_hat_svm, pos_label='PAIDOFF')

print('\nSVM: F1 score: ', svm_f1)
print('SVM: Jaccard score: ', svm_jac)

# Final LogRegression model
y_hat_regr = final_regr_model.predict(X_test)
regr_f1 = f1_score(y_test, y_hat_regr, average='weighted')
regr_jac = jaccard_score(y_test, y_hat_regr, pos_label='PAIDOFF')
y_hat_regr_prob = final_regr_model.predict_proba(X_test)
regr_ll = log_loss(y_test, y_hat_regr_prob)

print('\nLogisticRegression: F1 score: ', regr_f1)
print('LogisticRegression: Jaccard score: ', regr_jac)
print('LogisticRegression: Log Loss: ', regr_ll)

In [ ]:
# Prepare the final report, as described
    
report = {
            'Algorithm': ['KNN', 'Decistion Tree', 'SVM', 'LogisticRegression'],
            'Jaccard': [knn_jac, tree_jac, svm_jac, regr_jac],
            'F1-score': [knn_f1, tree_f1, svm_f1, regr_f1],
            'LogLoss': ['NA', 'NA', 'NA', regr_ll]
         }

final_report = pd.DataFrame(data=report, columns=['Algorithm', 'Jaccard', 'F1-score', 'LogLoss'], index=None)
# final_report.set_index('Algorithm', inplace=True)
final_report

# Report

You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Jaccard | F1-score | LogLoss |
| ------------------ | ------- | -------- | ------- |
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description                                                             |
| ----------------- | ------- | ------------- | ------------------------------------------------------------------------------ |
| 2020-10-27        | 2.1     | Lakshmi Holla | Made changes in import statement due to updates in version of  sklearn library |
| 2020-08-27        | 2.0     | Malika Singla | Added lab to GitLab                                                            |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
